In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
import sys
sys.path.append("/Workspace/Project_2/Databricks_Projects")
from validation_rules import not_null_rule, date_format_rule, concat_errors
from pyspark.sql.functions import col

In [0]:
df = spark.read.table("my_database.extracts.cln_smc_ln")

invalid_BillToSiteNumber = not_null_rule("BillToSiteNumber")
invalid_ShipToPartySiteNumber = not_null_rule("ShipToPartySiteNumber")
invalid_BillToAccountNumber = not_null_rule("BillToAccountNumber")
invalid_ShipToPartyNumber = not_null_rule("ShipToPartyNumber")
invalid_Service_offerings = not_null_rule("Service_offerings")
invalid_INVENTORY_ITEM_ID = not_null_rule("INVENTORY_ITEM_ID")
invalid_Billing_account = not_null_rule("Billing_account")
invalid_XX_OSS_Calendar_c = not_null_rule("XX_OSS_Calendar_c")

# Combine errors
Invalid_summary = concat_errors(invalid_BillToSiteNumber,invalid_ShipToPartySiteNumber,invalid_BillToAccountNumber,invalid_ShipToPartyNumber,invalid_Service_offerings, invalid_INVENTORY_ITEM_ID,invalid_Billing_account,invalid_XX_OSS_Calendar_c)

In [0]:
# Add error column(s) at the end of DataFrame
df_validated = (df.withColumn("invalid_BillToSiteNumber",invalid_BillToSiteNumber )
.withColumn("invalid_ShipToPartySiteNumber",invalid_ShipToPartySiteNumber )
.withColumn("invalid_BillToAccountNumber",invalid_BillToAccountNumber )
.withColumn("invalid_ShipToPartyNumber",invalid_ShipToPartyNumber )
.withColumn("invalid_Service_offerings",invalid_Service_offerings )
.withColumn("invalid_INVENTORY_ITEM_ID",invalid_INVENTORY_ITEM_ID )
.withColumn("invalid_Billing_account",invalid_Billing_account )
.withColumn("invalid_XX_OSS_Calendar_c",invalid_XX_OSS_Calendar_c )
.withColumn("Invalid_summary",Invalid_summary )
)

df_validated.write.mode("overwrite").saveAsTable("my_database.my_schema.All_SMC_Profile_ready")


In [0]:
# Split valid/invalid
df_valid = df_validated.filter(col("Invalid_summary") == "")
df_invalid = df_validated.filter(col("Invalid_summary") != "")

display(df_invalid)

In [0]:
# Write outputs (dummy table names)
df_valid.write.mode("overwrite").saveAsTable("my_database.my_schema.vld_SMC_valid")
df_invalid.write.mode("overwrite").saveAsTable("my_database.my_schema.vld_SMC_invalid")